#### Notebook example to run EMR serverless job from Sagemaker

us-east-1 applications: <br>
* pd-autoencoder-ad-v1 : 00f64bef5869kl09
* pd-autoencoder-ad-v2 : 00f66ohicnjchu09
* pd-autoencoder-ad-v3 : 00f6959e87i48609
<br>

us-west-2 applications: <br>
* pd-autoencoder-ad-container-v1: 00f672mqiak1fp0l


Note: while launching your job, please make note of the region from where you are running it.
jobs for us-east-region can only be launched from us-east-1 

In [2]:
pwd

'/root/eks-ml-pipeline'

In [3]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
  Using cached configparser-5.3.0-py3-none-any.whl (19 kB)
  Using cached pyspark-3.3.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)


In [4]:
pip install -r requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.11.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using cached boto3-1.26.30-py3-none-any.whl (132 kB)
  Using cached awswrangler-2.18.0-py3-none-any.whl (255 kB)
  Using cached fast_arrow-1.0.3-py3-none-any.whl (163 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached libclang-14.0.6-py2.py3-none-manylinux2010_x86_64.whl (14.1 MB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Using cached protobuf-3.19.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using c

In [ ]:
#pip install tensorflow
#import tensorflow

In [5]:
from eks_ml_pipeline import EMRServerless

In [6]:
# application id
application_id = '00f66ohicnjchu09'
# pass job_run_id only when a job is running else it can be empty
job_run_id = "00f6as23g5ha7509"
# don't chnage the serverless_job_role_arn
serverless_job_role_arn = 'arn:aws:iam::064047601590:role/hamza-emr-serverless-role'
# s3 bukcet name where the dependencies, logs and code sits
s3_bucket_name = 'emr-serverless-output-pd'
emr_emtry_point = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/s3_test_emr.py'
zipped_env_path = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/pyspark_deps_all_rec_types_v2.tar.gz'

In [11]:
emr_serverless = EMRServerless(application_id,job_run_id)

In [12]:
print("Starting EMR Serverless Spark App")
#emr_serverless.create_application("pd-autoencoder-test-emr-cli", "emr-6.6.0")
emr_serverless.start_application(application_id)
print(emr_serverless)

Starting EMR Serverless Spark App


KeyboardInterrupt: 

Below cell shows an example to run a new job

In [ ]:
# Run (and wait for) a Spark job
print("Submitting new Spark job")
job_run_id = emr_serverless.run_spark_job(
    script_location=emr_entry_point,
    job_role_arn=serverless_job_role_arn,
    application_id = application_id,
    arguments=[f"s3://{s3_bucket_name}/emr-serverless/output"],
    s3_bucket_name=s3_bucket_name,
    zipped_env_path = zipped_env_path
)

In [8]:
# use this to get the configuration of a job which has already run or running 
emr_serverless.get_job_run()

{'applicationId': '00f66ohicnjchu09',
 'jobRunId': '00f6as23g5ha7509',
 'name': '-clone',
 'arn': 'arn:aws:emr-serverless:us-east-1:064047601590:/applications/00f66ohicnjchu09/jobruns/00f6as23g5ha7509',
 'createdBy': 'arn:aws:sts::064047601590:assumed-role/AWSReservedSSO_DishTaasAdminDev_ea612f790bd52334/aakash.agrawal@dish.com',
 'createdAt': datetime.datetime(2022, 12, 14, 20, 53, 20, 120000, tzinfo=tzlocal()),
 'updatedAt': datetime.datetime(2022, 12, 14, 22, 48, 5, 746000, tzinfo=tzlocal()),
 'executionRole': 'arn:aws:iam::064047601590:role/hamza-emr-serverless-role',
 'state': 'FAILED',
 'stateDetails': "Job failed, please check complete logs in configured logging destination. ExitCode: 137. Last few exceptions: # java.lang.OutOfMemoryError: Java heap space\n2022-12-14 20:54:33.198704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file:

In [ ]:
# job_status = emr_serverless.get_job_run(job_run_id)
# print(f"Job finished: {job_run_id}, status is: {job_status.get('state')}")

# Fetch and print the logs
spark_driver_logs = emr_serverless.fetch_driver_log(s3_bucket_name)
print("File output from stdout.gz:\n----\n", spark_driver_logs, "\n----")

# Now stop and delete your application
#print("Stopping App")
#emr_serverless.stop_application()
#emr_serverless.delete_application()
print("Done! 👋")

In [34]:
emr_serverless.fetch_driver_log(s3_bucket_name)

File output from stdout.gz:
----
 2022-12-14 20:54:32.338517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 20:54:32.471433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native:/docker/usr/lib/hadoop/lib/native:/docker/usr/lib/hadoop-lzo/lib/native
2022-12-14 20:54:32.471462: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-14 20:54:33.198612: W tensorflow/compiler/xla/